# Initial Modelling V2

In [1]:
import cv2
import pandas as pd

final_data_path = open("step1.txt", "r")
final_data_pre = pd.read_csv(final_data_path.read().strip())

In [2]:
final_data_pre

,id,class_name,file_path,resolution
0,452f2473f3dd5817395dcaa16b58ab6e.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
1,2da7e04b024ef07fd40e4f84213d6ca7.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
2,708f00dd2a7c25d75680d18299d0254d.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
3,866d7784b51be1f26a96af4c99e73448.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
4,2e4fbf3ef0199cd4f664155171dbb849.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,1280x720
...,...,...,...,...
3821,2790a53203bb55d957f3d51eb8161592.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,730x548
3822,d8891a6e393c829c349241f4baac8ad5.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,800x378
3823,244c532e45a81497e3d51e81a1fcb04d.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,872x534
3824,8109576227fc9d3f8e5b74db76b4da70.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,480x360


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np


final_data_pre = final_data_pre[final_data_pre["class_name"].str.contains("banana") & ~final_data_pre["class_name"].str.contains("google")]

label_processor = tf.keras.layers.StringLookup(
    output_mode='int', vocabulary=final_data_pre['class_name'].unique(),
    num_oov_indices=0,
    mask_token=None
)

def load_image(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    return img, label


# --- 4. ASSEMBLE THE DATASET ---

# Create dataset from DataFrame columns
dataset = tf.data.Dataset.from_tensor_slices((final_data_pre['file_path'].values, final_data_pre['class_name'].values))
# Convert string to int
dataset = dataset.map(lambda x, y: (x, label_processor(y)))
# Load the image using tensorflow as an image array
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)


2025-12-16 09:22:11.371476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1765876933.733459 2059581 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1212 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
%run modeldef2.py

# --- Configuration ---
NUM_CLASSES = len(final_data_pre['class_name'].unique())

# Create the Unified Model
model = UnifiedResNetClassifier(num_classes=NUM_CLASSES)
model.summary()

# Compile
# We use from_logits=True because the final layer has no Softmax
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(dataset, epochs=20)

Model: "unified_res_net_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

...

1/6 ━━━━━━━━━━━━━━━━━━━━ 1:24 17s/step - accuracy: 0.0000e+00 - loss: 2.3806

I0000 00:00:1765876960.320887 2059667 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.0162 - loss: 2.3123

2025-12-16 09:22:42.399232: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5099', 4 bytes spill stores, 4 bytes spill loads

2025-12-16 09:22:44.387477: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1137 - loss: 2.1815 
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.1983 - loss: 1.9872 
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.2187 - loss: 1.9222
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.2653 - loss: 1.8828
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.2711 - loss: 1.8502
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.3003 - loss: 1.8272
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.3149 - loss: 1.8047
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.3382 - loss: 1.7766
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.4023 - loss: 1.7388
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.4227 - loss: 1.6982 
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.4490 - loss: 1.6649
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.4869 - loss: 1.6341
Epoch 13/2

### As seen in the epoch results, the model isnt performing too well, with an accuracy of 0.548 and a loss of 1.49, it seems to be quite high. So for the next iteration of the processing pipeline, more augmentations and hyperparameter optimizations will be done to increase the model's accuracy.